In [1]:
from typing import List

from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph

from reflexion_chains import revisor_chain, first_responder_chain
from execute_tools import execute_tools

In [2]:
DRAFT = "draft"
TOOLS = "execute_tools"
REVISOR = "revisor"

MAX_ITERATIONS = 2

In [4]:
def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return TOOLS

In [5]:
graph = MessageGraph()

graph.add_node(DRAFT, first_responder_chain)
graph.add_node(TOOLS, execute_tools)
graph.add_node(REVISOR, revisor_chain)

graph.set_entry_point(DRAFT)
graph.add_edge(DRAFT, TOOLS)
graph.add_edge(TOOLS, REVISOR)
graph.add_conditional_edges(REVISOR, event_loop)

In [6]:
app = graph.compile()

print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor -.-> draft;
	revisor -.-> execute_tools;
	revisor -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

              +-----------+      
              | __start__ |      
              +-----------+      
                    *            
                    *            
                    *            
                +-------+        
                | draft |        
                +-------+        
             ***         ...     
            *               .    
          **                 ... 
+---------------+               .
| execute_tools |            ... 
+---------------+           .    
             ***         ...     
        

In [7]:
response = app.invoke(
    "Write me a blog post on how small business can leverage AI to grow"
)

response

[HumanMessage(content='Write me a blog post on how small business can leverage AI to grow', additional_kwargs={}, response_metadata={}, id='35c4af3e-b9ca-45ae-90a6-3dbab1f4b5b7'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 469, 'prompt_tokens': 228, 'total_tokens': 697, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcmeUdFiki9oFrLBIGecbYnh6enMm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--307de6e4-d85e-47d4-9482-12384113d772-0', tool_calls=[{'name': 'AnswerQuestion', 'args': {'answer': 'Small businesses have a significant opportunity to leverage artificial intelligence (AI) to drive growth and im

In [8]:
response[-1].tool_calls[0]["args"]["answer"]

'Small businesses can accelerate growth in 2025 by strategically leveraging artificial intelligence (AI) to enhance operations and customer engagement. Here’s how:\n\n1. **Personalized Marketing & Customer Support:** AI tools like Jasper AI enable quick creation of targeted content, while chatbots such as Tidio offer 24/7 customer support with minimal human intervention. HubSpot’s free CRM tier integrates AI to automate marketing and sales workflows effectively.\n\n2. **Sales Forecasting & Inventory Management:** AI-driven analytics provide accurate demand forecasting, enabling better inventory decisions and cost savings.\n\n3. **Automation of Routine Tasks:** Tools like Otter.ai automate meeting transcriptions and note-taking, saving valuable time. Other automations include invoicing, scheduling, and data entry.\n\n4. **Data-Driven Decision Making:** AI analytics help uncover customer behaviors and market trends, empowering informed strategic choices.\n\n5. **Social Media & Content Op

In [ ]:
response[-1].tool_calls[0]#["args"]

{'name': 'ReviseAnswer',
 'args': {'answer': 'Small businesses can accelerate growth in 2025 by strategically leveraging artificial intelligence (AI) to enhance operations and customer engagement. Here’s how:\n\n1. **Personalized Marketing & Customer Support:** AI tools like Jasper AI enable quick creation of targeted content, while chatbots such as Tidio offer 24/7 customer support with minimal human intervention. HubSpot’s free CRM tier integrates AI to automate marketing and sales workflows effectively.\n\n2. **Sales Forecasting & Inventory Management:** AI-driven analytics provide accurate demand forecasting, enabling better inventory decisions and cost savings.\n\n3. **Automation of Routine Tasks:** Tools like Otter.ai automate meeting transcriptions and note-taking, saving valuable time. Other automations include invoicing, scheduling, and data entry.\n\n4. **Data-Driven Decision Making:** AI analytics help uncover customer behaviors and market trends, empowering informed strateg